In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

In [3]:
import os
os.chdir(r"C:\Users\namra\Downloads\Cataract-Detection-and-Classification-master\Cataract-Detection-and-Classification-master\phase 2 Types\Deep learning\model_40")


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#Training and testing directories
train = r'C:\Users\namra\Downloads\Cataract-Detection-and-Classification-master\Cataract-Detection-and-Classification-master\phase 2 Types\Deep learning\preprocessed_images_40\TRAIN_DIR/'
test = r'C:\Users\namra\Downloads\Cataract-Detection-and-Classification-master\Cataract-Detection-and-Classification-master\phase 2 Types\Deep learning\preprocessed_images_40\TEST_DIR/'

#Augumentation with only zoom and grayscale format

train_datagen2 = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip = True,
                             zoom_range = 0.3,
                             rescale=1/255.0
                            )

#Corresponding generator
train_generator2 = train_datagen2.flow_from_directory(
    train,
    color_mode='grayscale',
    target_size = (128,128),
    batch_size=32,
    class_mode = 'categorical',
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen2 = ImageDataGenerator(
     rescale=1/255.0 )
#Using testing data as validation data

validation_generator2 = test_datagen2.flow_from_directory(
    test,
    color_mode='grayscale',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42 )

x_train,y_train = next(train_generator2)
x_test,y_test = next(validation_generator2)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 1)
(32, 128, 128, 1)


In [5]:
#Basic CNN Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
import numpy as np
from keras.callbacks import ModelCheckpoint

Mobile Net **V2**

In [12]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop

In [18]:
#Training and testing directories
# train = 'TRAIN_DIR/'
# test = 'TEST_DIR/'
train = r'C:\Users\namra\Downloads\Cataract-Detection-and-Classification-master\Cataract-Detection-and-Classification-master\phase 2 Types\Deep learning\preprocessed_images_40\TRAIN_DIR/'
test = r'C:\Users\namra\Downloads\Cataract-Detection-and-Classification-master\Cataract-Detection-and-Classification-master\phase 2 Types\Deep learning\preprocessed_images_40\TEST_DIR/'

#For RGB images
train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                             preprocessing_function = preprocess_input)

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (128,128),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (128,128),
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 3)
(32, 128, 128, 3)


In [24]:
# mobilenet_weight_path = 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'

# mobileNet = MobileNetV2(
#     input_shape = (128,128,3),
#     alpha=1.0,
#     include_top=False,
#     weights= mobilenet_weight_path
# )

# MN_model = Sequential()
# MN_model.add(mobileNet)
# MN_model.add(layers.GlobalAveragePooling2D())
# MN_model.add(layers.Dropout(0.5))
# MN_model.add(layers.Dense(3, activation='softmax'))
    
# MN_model.compile(
#         loss='categorical_crossentropy',
#         optimizer=Adam(lr=0.00002),
#         metrics=['accuracy']
#     )

# MN_model.summary()

**SQUEEZENET**

In [15]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.7.0


In [19]:
bnmomemtum=0.9
def fire(x, squeeze, expand):
    y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
    y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
    y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
    return tf.keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)

x = tf.keras.layers.Input(shape=[128,128, 3]) # input is 192x192 pixels RGB

y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(64, 128)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(3, activation='softmax')(y)

model = tf.keras.Model(x, y)
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 128, 128, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_22 (BatchN  (None, 128, 128, 32  128        ['conv2d_20[0][0]']              
 ormalization)                  )                                                           

In [20]:
x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

(32, 128, 128, 3)
(32, 128, 128, 3)


In [21]:
#for fitting the model
from keras.callbacks import EarlyStopping
EPOCHS_SQ = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

SQ_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_SQ,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[SQ_STOP]
)

Epoch 1/30
12/12 [==============================] - 16s 1s/step - loss: 0.8531 - accuracy: 0.6323 - val_loss: 0.9772 - val_accuracy: 0.4141
Epoch 2/30
12/12 [==============================] - 16s 1s/step - loss: 0.4633 - accuracy: 0.8995 - val_loss: 0.8697 - val_accuracy: 0.5938
Epoch 3/30
12/12 [==============================] - 15s 1s/step - loss: 0.3164 - accuracy: 0.9127 - val_loss: 0.4836 - val_accuracy: 0.8516
Epoch 4/30
12/12 [==============================] - 15s 1s/step - loss: 0.2099 - accuracy: 0.9444 - val_loss: 0.3178 - val_accuracy: 0.8984
Epoch 5/30
12/12 [==============================] - 16s 1s/step - loss: 0.1523 - accuracy: 0.9788 - val_loss: 0.3233 - val_accuracy: 0.9219
Epoch 6/30
12/12 [==============================] - 15s 1s/step - loss: 0.1251 - accuracy: 0.9815 - val_loss: 0.3996 - val_accuracy: 0.8750
Epoch 7/30
12/12 [==============================] - 15s 1s/step - loss: 0.1336 - accuracy: 0.9788 - val_loss: 0.4000 - val_accuracy: 0.8359
Epoch 8/30
12/12 [==

In [23]:
model.save('spq_model.h5')

In [29]:
from keras.preprocessing import image

In [33]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(128, 128))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

    

In [34]:
from keras.models import load_model
sd = load_model('spq_model.h5')
img_path = r'C:\Users\namra\Downloads\Cataract-Detection-and-Classification-master\Cataract-Detection-and-Classification-master\phase 2 Types\Deep learning\preprocessed_images_40\TEST_DIR\severe\severe_165.png'    # dog
    #img_path = '/media/data/dogscats/test1/19.jpg'      # cat

    # load a single image
new_image = load_image(img_path)

# check prediction
pred = sd.predict(new_image)

In [37]:
max(pred[0])

0.83219814